# 引入所需要模块

In [4]:
import os
import random
import pandas as pd
import numpy as np
from PIL import Image

# 加载数据集

In [5]:
root_dir=os.path.abspath('E:/data/age')
train=pd.read_csv(os.path.join(root_dir,'train.csv'))
test=pd.read_csv(os.path.join(root_dir,'test.csv'))

In [8]:
#随机读取图片
i=random.choice(train.index)
img_name=train.ID[i]
print(img_name)
img=Image.open(os.path.join(root_dir,'Train',img_name))
img.show()
print(train.Class[i])

10471.jpg
MIDDLE


# 格式化图片尺寸和将数据存到numpy数组

In [10]:
temp=[]
for img_name in train.ID:
    img_path=os.path.join(root_dir,'Train',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
train_x=np.stack(temp)
print(train_x.shape)
print(train_x.ndim)

(19906, 32, 32, 3)
4


In [12]:
temp=[]
for img_name in test.ID:
    img_path=os.path.join(root_dir,'Test',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
test_x=np.stack(temp)
print(test_x.shape)

(6636, 32, 32, 3)


In [13]:
train_x = train_x / 255.
test_x = test_x / 255.

In [15]:
train.Class.value_counts(normalize=True)

MIDDLE    0.542751
YOUNG     0.336883
OLD       0.120366
Name: Class, dtype: float64

In [16]:
test['Class'] = 'MIDDLE'
test.to_csv('sub01.csv', index=False)

# 创建模型

In [19]:
import keras
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
train_y=lb.fit_transform(train.Class)
print(train.Class)
print(train_y)
train_y=keras.utils.np_utils.to_categorical(train_y)
print(train_y)
print(train_y.shape)

0        MIDDLE
1         YOUNG
2        MIDDLE
3         YOUNG
4        MIDDLE
5        MIDDLE
6         YOUNG
7         YOUNG
8         YOUNG
9           OLD
10        YOUNG
11       MIDDLE
12       MIDDLE
13       MIDDLE
14       MIDDLE
15       MIDDLE
16          OLD
17        YOUNG
18          OLD
19       MIDDLE
20        YOUNG
21          OLD
22        YOUNG
23       MIDDLE
24       MIDDLE
25        YOUNG
26       MIDDLE
27          OLD
28       MIDDLE
29        YOUNG
          ...  
19876       OLD
19877       OLD
19878     YOUNG
19879     YOUNG
19880     YOUNG
19881    MIDDLE
19882    MIDDLE
19883     YOUNG
19884       OLD
19885     YOUNG
19886    MIDDLE
19887    MIDDLE
19888     YOUNG
19889     YOUNG
19890    MIDDLE
19891     YOUNG
19892    MIDDLE
19893    MIDDLE
19894    MIDDLE
19895    MIDDLE
19896     YOUNG
19897    MIDDLE
19898       OLD
19899    MIDDLE
19900    MIDDLE
19901    MIDDLE
19902     YOUNG
19903    MIDDLE
19904    MIDDLE
19905    MIDDLE
Name: Class, Length: 199

In [11]:
#构建神经网络
input_num_units=(32,32,3)
hidden_num_units=500
output_num_units=3
epochs=5
batch_size=128

In [36]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,InputLayer
model=Sequential({
    InputLayer(input_shape=input_num_units),
    Flatten(),
    Dense(units=hidden_num_units,activation='relu'),
    Dense(input_shape=(32,32,3),units=output_num_units,activation='softmax')
})
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 500)               1536500   
_________________________________________________________________
dense_46 (Dense)             (None, 3)                 1503      
Total params: 1,538,003
Trainable params: 1,538,003
Non-trainable params: 0
_________________________________________________________________


# 编译模型

In [37]:
# model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/5
19906/19906 [==============================] - 4s - loss: 0.8878 - acc: 0.5809     
Epoch 2/5
19906/19906 [==============================] - 4s - loss: 0.8420 - acc: 0.6077     
Epoch 3/5
19906/19906 [==============================] - 4s - loss: 0.8210 - acc: 0.6214     
Epoch 4/5
19906/19906 [==============================] - 4s - loss: 0.8149 - acc: 0.6194     
Epoch 5/5
19906/19906 [==============================] - 4s - loss: 0.8042 - acc: 0.6305     


In [38]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.2)

Train on 15924 samples, validate on 3982 samples
Epoch 1/5
15924/15924 [==============================] - 3s - loss: 0.7970 - acc: 0.6375 - val_loss: 0.7854 - val_acc: 0.6396
Epoch 2/5
15924/15924 [==============================] - 3s - loss: 0.7919 - acc: 0.6378 - val_loss: 0.7767 - val_acc: 0.6519
Epoch 3/5
15924/15924 [==============================] - 3s - loss: 0.7870 - acc: 0.6404 - val_loss: 0.7754 - val_acc: 0.6534
Epoch 4/5
15924/15924 [==============================] - 3s - loss: 0.7806 - acc: 0.6439 - val_loss: 0.7715 - val_acc: 0.6524
Epoch 5/5
15924/15924 [==============================] - 3s - loss: 0.7755 - acc: 0.6519 - val_loss: 0.7970 - val_acc: 0.6346


# 使用卷积神经网络

In [20]:
#参数初始化
filters=10
filtersize=(5,5)

epochs =5
batchsize=128

input_shape=(32,32,3)

In [27]:
from keras.models import Sequential
model = Sequential()

model.add(keras.layers.InputLayer(input_shape=input_shape))

model.add(keras.layers.convolutional.Conv2D(filters, filtersize, strides=(1, 1), padding='valid', data_format="channels_last", activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=3, input_dim=50,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=epochs, batch_size=batchsize,validation_split=0.3)

model.summary()

Train on 13934 samples, validate on 5972 samples
Epoch 1/5
13934/13934 [==============================] - 16s - loss: 0.9239 - acc: 0.5642 - val_loss: 0.8573 - val_acc: 0.5996
Epoch 2/5
13934/13934 [==============================] - 16s - loss: 0.8234 - acc: 0.6255 - val_loss: 0.7915 - val_acc: 0.6445
Epoch 3/5
13934/13934 [==============================] - 15s - loss: 0.7884 - acc: 0.6467 - val_loss: 0.7713 - val_acc: 0.6621
Epoch 4/5
13934/13934 [==============================] - 15s - loss: 0.7723 - acc: 0.6578 - val_loss: 0.7769 - val_acc: 0.6567
Epoch 5/5
13934/13934 [==============================] - 15s - loss: 0.7526 - acc: 0.6656 - val_loss: 0.7442 - val_acc: 0.6718
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 10)     

# 输出结果

In [28]:
pred=model.predict_classes(test_x)
pred=lb.inverse_transform(pred)
print(pred)
test['Class']=pred
test.to_csv('sub02.csv',index=False)

6636/6636 [==============================] - 3s     
['MIDDLE' 'YOUNG' 'MIDDLE' ..., 'MIDDLE' 'MIDDLE' 'MIDDLE']


In [31]:
i = random.choice(train.index)
img_name = train.ID[i]

img=Image.open(os.path.join(root_dir,'Train',img_name))
img.show()
pred = model.predict_classes(train_x)
print('Original:', train.Class[i], 'Predicted:', lb.inverse_transform(pred[i]))

19872/19906 [============================>.] - ETA: 0sOriginal: MIDDLE Predicted: MIDDLE
